In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import time
from sqlalchemy import inspect, create_engine
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# HOSTNAME = "localhost"
# PORT = 5432
# DATABASE = "diapers_db"
# USERNAME = "postgres"
# PASSWORD = "password"
# path = f'postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}:{PORT}/{DATABASE}'
# engine = create_engine(path)

In [4]:
# conn = engine.connect()

In [5]:
#Website searching Data Analyst positions
indeed = "https://www.indeed.com/jobs?q=data+analyst"

browser.visit(indeed)



In [6]:
html = browser.html
html[0:500]

'<html dir="ltr" lang="en" class="js-focus-visible" data-js-focus-visible=""><head>\n    <link rel="shortcut icon" href="/images/favicon.ico">\n    <title>Data Analyst Jobs, Employment | Indeed.com</title>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8">\n    <meta name="description" content="28,371 Data Analyst jobs available on Indeed.com. Apply to Data Analyst, Entry Level Data Analyst, Senior Data Analyst and more!">\n    <meta name="keywords" content="Data Analyst Jobs, Em'

In [7]:
soup = BeautifulSoup(browser.html, "html.parser")
#print(soup.prettify())

In [8]:
soup= BeautifulSoup(browser.html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [9]:
#Find code to loop through web page and pull the title of the job and possibly the location, then find code to click the job and acquire the salary.
jobs = soup.find_all('div', {'class': 'cardOutline'})
len(jobs)


15

In [10]:
#Find the jobs titles for positions and the link IF available
job = jobs[0]
title = job.find('a', {"class": 'jcs-JobTitle'}).text
link = "www.indeed.com" + job.find('a', {"class": 'jcs-JobTitle'})["href"]


In [11]:
#Find company name and the link IF available
company = job.find('a', {"class": 'companyOverviewLink'}).text
company_link = "www.indeed.com" + job.find('a', {"class": 'companyOverviewLink'})["href"]

In [12]:
#Find the rating for the company/job
rating = job.find('span', {"class": 'ratingNumber'}).text
rating

'3.4'

In [13]:
#Find the location of the job listing
loc = job.find('div', {"class": 'companyLocation'}).text
loc

'Hybrid remote in Norfolk, VA 23502'

In [14]:
#Find the salary for the job listing
salary = job.find('div', {"class": 'salary-snippet-container'}).text
salary

'$50 - $75 an hour'

In [15]:
#Pull the text only from the parent for job salary
job.find('svg', {"aria-label": "Job type"}).parent.text

'Full-time +1'

In [18]:
#
rows = []
browser.visit(indeed)
for i in range(1, 15):
    
    html = browser.html
    soup = BeautifulSoup(browser.html, "html.parser")
    jobs = soup.find_all('div', {'class': 'cardOutline'})
    
    
    for job in jobs:
        title = job.find('a', {"class": 'jcs-JobTitle'}).text
        link = "www.indeed.com" + job.find('a', {"class": 'jcs-JobTitle'})["href"]

        company_name = job.find('span', {'class': 'companyName'})
        if company_name: 
            company = company_name.text
        else:
            company = None
        
        company_con = job.find('a', {"class": 'companyOverviewLink'})
        if company_con: 
            company_link = "www.indeed.com" + company_con["href"]
        else:
            company_link = None

        rating_con = job.find('span', {"class": 'ratingNumber'})
        if rating_con: 
            rating = rating_con.text
        else:
            rating = None


        loc = job.find('div', {"class": 'companyLocation'}).text

        salary_con = job.find('div', {"class": 'salary-snippet-container'})
        if salary_con: 
            salary = salary_con.text
        else:
            salary = None


        status_con = job.find('svg', {"aria-label": "Job type"})
        if status_con:
            status = status_con.parent.text
        else:
            status = None

        data = {}
        data["job_title"] = title
        data["job_link"] = link
        data["company"] = company
        data["company_link"] = company_link
        data["company_rating"] = rating
        data["location"] = loc
        data["salary"] = salary
        data["status"] = status
        rows.append(data)
    
    browser.links.find_by_text(f"{i+1}").click()
        
df = pd.DataFrame(rows)
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,status
0,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Ad...,Uline,www.indeed.com/cmp/Uline,3.1,"Pleasant Prairie, WI 53158",None,None
1,Board Certified Behavior Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Db...,Infinity Counseling Group,www.indeed.com/cmp/Infinity-Counseling-Group,3.4,"Hybrid remote in Norfolk, VA 23502",$50 - $75 an hour,Full-time +1
2,Business Intelligence Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0DS...,"iClassPro, Inc.",None,None,"Hybrid remote in Longview, TX 75605","$55,000 - $98,757 a year",Full-time
3,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BU...,CardWorks,www.indeed.com/cmp/Cardworks-Servicing,3.3,"South Jordan, UT 84095","From $59,000 a year",Full-time
4,IT Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0De...,AFR Services,None,None,"Hybrid remote in Plano, TX 75093",None,Full-time


In [19]:
len(df)

210

In [22]:
df["company_rating"] = pd.to_numeric(df.company_rating)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       210 non-null    object 
 1   job_link        210 non-null    object 
 2   company         210 non-null    object 
 3   company_link    140 non-null    object 
 4   company_rating  140 non-null    float64
 5   location        210 non-null    object 
 6   salary          121 non-null    object 
 7   status          158 non-null    object 
dtypes: float64(1), object(7)
memory usage: 13.2+ KB


In [23]:
df.status.value_counts()

Full-time       86
Contract        47
Full-time +1    14
Full-time +2     5
Contract +1      2
Temporary +1     2
Part-time        1
Internship       1
Name: status, dtype: int64

In [26]:
#combine with regex s=space d=catch or match all numbers \+ = plus sign matches plus sign, regex meaning.
df["status"] = df.status.str.replace("\s\+\d","")

C:\Users\19722\AppData\Local\Temp\ipykernel_5680\152645899.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["status"] = df.status.str.replace("\s\+\d","")


In [27]:
df.status.value_counts()

Full-time     105
Contract       49
Temporary       2
Part-time       1
Internship      1
Name: status, dtype: int64

In [28]:
#Status column -  ID, name, last_updated
df.status.unique()

array([None, 'Full-time', 'Contract', 'Part-time', 'Internship',
       'Temporary'], dtype=object)

In [30]:
#Status lookup table
status = df.status.dropna().unique()
status_df = pd.DataFrame()
status_df["status"] = status
status_df

,status
0,Full-time
1,Contract
2,Part-time
3,Internship
4,Temporary


In [36]:
#
companies = df.loc[:,["company","company_link", "company_rating"]].drop_duplicates().reset_index(drop=True)
companies.company.value_counts()

Insight Global                                 2
Uline                                          1
Clairvoyant                                    1
Envision                                       1
Halliburton                                    1
                                              ..
Prysmian Group                                 1
US Centers for Medicare & Medicaid Services    1
Broadpath                                      1
Tesla                                          1
Tata Technologies Inc.                         1
Name: company, Length: 192, dtype: int64

In [37]:
companies.loc[companies.company == "Insight Global"]

,company,company_link,company_rating
32,Insight Global,www.indeed.com/cmp/Insight-Global,3.9
89,Insight Global,None,NaN


In [38]:
df.loc[df.company == "Insight Global", "company_link"] = "www.indeed.com/cmp/Insight-Global"
df.loc[df.company == "Insight Global", "company_rating"] = 3.9

In [39]:
companies = df.loc[:,["company","company_link", "company_rating"]].drop_duplicates().reset_index(drop=True)
companies.company.value_counts()

Uline                                          1
Infinity Counseling Group                      1
Larsen and Toubro (L&T)                        1
Envision                                       1
Halliburton                                    1
                                              ..
Razorfish                                      1
Prysmian Group                                 1
US Centers for Medicare & Medicaid Services    1
Broadpath                                      1
Tata Technologies Inc.                         1
Name: company, Length: 192, dtype: int64

In [32]:
#State look-up table
state = df.State.dropna().unique()
state_df = pd.DataFrame()
state_df["state"] = state
state_df

AttributeError: 'DataFrame' object has no attribute 'State'

In [21]:
df.head(20)

,job_title,job_link,company,company_link,company_rating,location,salary,status
0,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Ad...,Uline,www.indeed.com/cmp/Uline,3.1,"Pleasant Prairie, WI 53158",None,None
1,Board Certified Behavior Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Db...,Infinity Counseling Group,www.indeed.com/cmp/Infinity-Counseling-Group,3.4,"Hybrid remote in Norfolk, VA 23502",$50 - $75 an hour,Full-time +1
2,Business Intelligence Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0DS...,"iClassPro, Inc.",None,None,"Hybrid remote in Longview, TX 75605","$55,000 - $98,757 a year",Full-time
3,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BU...,CardWorks,www.indeed.com/cmp/Cardworks-Servicing,3.3,"South Jordan, UT 84095","From $59,000 a year",Full-time
4,IT Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0De...,AFR Services,None,None,"Hybrid remote in Plano, TX 75093",None,Full-time
5,Healthcare Analyst II - Network Development Ma...,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Cb...,Fallon Health,www.indeed.com/cmp/Fallon-Health,3.1,"Hybrid remote in Worcester, MA 01608",None,Full-time
6,"All-Source Analyst, Mid",www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Ca...,Booz Allen Hamilton,www.indeed.com/cmp/Booz-Allen-Hamilton,3.9,"Charlottesville, VA",None,None
7,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0DS...,The World's Largest Social Network,None,None,"Remote in San Diego, CA",$38 - $41 an hour,Full-time +1
8,Sr Business Data Analyst- Hybrid,www.indeed.com/company/Apisero/jobs/Senior-Bus...,Apisero,None,None,"Hybrid remote in Brenham, TX 77833",$70 - $75 an hour,Contract
9,Data Analyst,www.indeed.com/company/COMTEC-INFORMATION-SYST...,ComTec information system,www.indeed.com/cmp/Comtec-Information-Systems,3.7,"Plano, TX 75023",$40 - $47 an hour,Full-time +1


In [ ]:
#
df.info()


In [ ]:
#
df.location.value_counts()

In [ ]:
#
def getState(loc):
    if "," in loc:
        state = loc.replace("\xa0", " ").split(",")[1].strip().split(" ")[0].strip()
    else:
        state = None
        
    return(state)
        

In [ ]:
#
getState("Austin, TX (Downtown area)")

In [ ]:
#
df["state"] = df.location.apply(lambda x: getState(x))
df.info()

In [ ]:
#
df.state.value_counts()

In [ ]:
#
df.loc[df.location.str.contains("\(Downtown")]

In [ ]:
df.iloc[71].location

In [ ]:
#
def getRemote(loc):
    if "remote" in loc.lower():
        return(True)
    else:
        return(False)
        
  

In [ ]:
#
df["is_remote"] = df.location.apply(lambda x: getRemote(x))
df.info()

In [ ]:
df.is_remote.value_counts()

In [ ]:
#Don't worry about it
def getCity(loc):
    if "," in loc:
        city_list = loc.replace("\xa0", " ").split(",")[0].strip().split(" ")
        city = city_list[0] if len(city_list) ==1 else city_list[1]
    else:
        city = None
        
    return(city)

In [ ]:
df["city"] = df.location.apply(lambda x: getCity(x))
df.info()

In [ ]:
#
df.city.value_counts()

In [ ]:
#
df.loc[df.location.str.contains("in")]

In [ ]:
#Set-up sleep function
# browser.visit(dice_lyst)
# time.sleep(2)

In [ ]:
def getAd(link):
    if "pagead" in link.lower():
        return(True)
    else:
        return(False)

In [ ]:
df["is_ad"] = df.job_link.apply(lambda x: getAd(x))
df.info()

In [ ]:
df.is_ad.value_counts()

In [ ]:
#Paying for Ads, add to main job table as bool
df.groupby("company").is_ad.sum().sort_values(ascending=False)